! jt -r

In [330]:
from importlib import reload
reload(aux)
reload(con)
reload(dt)
reload(info)

<module 'src.info' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\info.py'>

In [331]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime,date
import src.info as info
import src.in_out as io
import numpy as np
from src.density import Density, draw
import itertools

# Hazırlık Aşaması

**data** klasöründeki tüm dosya yolları alınıyor
```python
>>> path_list
Out []:
 ['C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6C_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6C_U8.xlsx']
```

In [2]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

Ürün adları sırayla alınıyor
```py
>>> product_names
Out[]:
['6AU8', '6BU8', '6CU8', '6AU8', '6BU8', '6CU8']```

In [3]:
product_names = [info.get_productName(path) for path in path_list]

Tarihler sırayla alınıyor
```py
>>> date_list
Out[]:
[datetime.date(2018, 9, 6),
 datetime.date(2018, 9, 6),
 datetime.date(2018, 9, 6),
 datetime.date(2018, 9, 7),
 datetime.date(2018, 9, 7),
 datetime.date(2018, 9, 7)]
```

In [4]:
date_list = [info.get_productDate(path) for path in path_list]

Hangi sütunlardaki verileri alacağımızı aşağıdaki listede belirityoruz

In [5]:
cols= ["Time","BID price","ASK price"]

Dosyalardan veriler dataframe olarak okunuyor ve listeye atılıyor.
Okunan_veriler'in yapısı: <br/>
```py
list(pd.DataFrame,pd.DataFrame.....)
```

In [6]:
okunan_veriler = [pd.read_excel(path,usecols=cols) for path in path_list]

Her bir DataFrame'deki sütunlar yeniden adlandırılıyor

In [7]:
columns=['time','bid_price','ask_price']

In [8]:
for df in okunan_veriler:
    df.columns = columns

In [53]:
time_stamp = okunan_veriler[0].time

# Mid Price Hesaplama ve Saatlik Bölme 

hesaplanan her bir mid_price **list_of_mid_price** adlı listeye atılıyor. Listenin yapısı aşağıdaki gibidir:
```py
list(pd.Series,pd.Series,pd.Series...)
```

In [9]:
list_of_mid_price = list(map(info.get_mid_price,okunan_veriler))

Bulunan mid price hiyerarşik indeksleme DataFrame'de birleştiriliyor.

In [17]:
arrays = [date_list,product_names]

In [18]:
df_mp = pd.DataFrame(data=list_of_mid_price, index = arrays)

Bu dataframe zaman sütunu eklenip saatlik olarak bölünüyor.

In [142]:
df_mp_t = df_mp.T

In [144]:
df_mp_t['time'] = time_stamp

```py
>>> len(list_of_hourly_mp)
Out[]: 23
>>> df.columns.levels
Out[]:FrozenList([[2018-09-06 00:00:00, 2018-09-07 00:00:00, 'time'], ['6AU8', '6BU8', '6CU8', '']])
```

In [181]:
list_of_hourly_mp = info.split_df(df_mp_t,23)

----

# Ürünler Eşleştiriliyor

```python
>>> non_duplicate_dates
Out[]:[datetime.date(2018, 9, 6), datetime.date(2018, 9, 7)]
>>> non_duplicate_names
Out[]:['6AU8', '6BU8', '6CU8']
>>> paired_products
Out[]: [('6AU8', '6BU8'), ('6AU8', '6CU8'), ('6BU8', '6CU8')]
```

In [13]:
non_duplicate_dates = list(dict.fromkeys(date_list)) #tekrar eden tarihler kaldırıldı.

In [14]:
non_duplicate_names = list(dict.fromkeys(product_names)) #tekrar eden ürünler kaldırıldı.

In [15]:
paired_products = list(itertools.combinations(non_duplicate_names, 2)) #2'li kombinasyon kullanılarak ürünler eşleştiriliyor.

----

```py
>>> 
Out[]:

```

# Spread Hesaplanıyor

**list_of_hourly_mp** da hiyerarşi şu şekildedir: <br/>
* Önce saate erişiriz
* İkinci olarak tarihe erişiriz
* Son olarak ürüne erişiriz <br/>
**Buna göre spread hesaplamak için aşağıdaki yol izlenir:** <br/>
    1. Her bir saate eriş
    2. Her bir tarihe eriş
    3. Pairleri al
    4. *spread_hesapla(pair): return pd.Series*
    

In [329]:
birlestirilmis = []
for hour in range(23):
    first_df = list_of_hourly_mp[hour]
    first_df= first_df.set_index(first_df.time,drop=True)
    for date in non_duplicate_dates:
        df = first_df[pd.Timestamp(date)]
        for pair in paired_products:
            pair_1 = df[pair[0]]
            pair_2 = df[pair[1]]
            spread = aux.find_spread(pair_1,pair_2,values)            
            birlestirilmis.append(dict(pair1=pair_1,pair2=pair_2,date=date,spread = spread))
            

size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600
i: 3599
size: 3600
3600


In [349]:
birlestirilmis[0]

{'pair1': time
 18:00:00    0.71930
 18:00:01    0.71930
 18:00:02    0.71925
 18:00:03    0.71925
 18:00:04    0.71925
 18:00:05    0.71935
 18:00:06    0.71935
 18:00:07    0.71935
 18:00:08    0.71935
 18:00:09    0.71935
 18:00:10    0.71935
 18:00:11    0.71935
 18:00:12    0.71935
 18:00:13    0.71935
 18:00:14    0.71935
 18:00:15    0.71935
 18:00:16    0.71935
 18:00:17    0.71935
 18:00:18    0.71935
 18:00:19    0.71935
 18:00:20    0.71935
 18:00:21    0.71935
 18:00:22    0.71935
 18:00:23    0.71935
 18:00:24    0.71935
 18:00:25    0.71935
 18:00:26    0.71935
 18:00:27    0.71935
 18:00:28    0.71935
 18:00:29    0.71935
              ...   
 18:59:30    0.71925
 18:59:31    0.71925
 18:59:32    0.71925
 18:59:33    0.71925
 18:59:34    0.71925
 18:59:35    0.71925
 18:59:36    0.71925
 18:59:37    0.71925
 18:59:38    0.71925
 18:59:39    0.71925
 18:59:40    0.71925
 18:59:41    0.71925
 18:59:42    0.71925
 18:59:43    0.71925
 18:59:44    0.71925
 18:59:45    0.7192

In [325]:
list_of_hourly_mp[2].set_index(list_of_hourly_mp[2].time)

2018-09-06 00:00:00                   2018-09-07 00:00:00           \
                        6AU8     6BU8     6CU8                6AU8     6BU8   
time                                                                          
20:00:00             0.71955  1.29185  0.71955             0.71955  1.29185   
20:00:01             0.71955  1.29185  0.71955             0.71955  1.29185   
20:00:02             0.71955  1.29185  0.71955             0.71955  1.29185   
20:00:03             0.71955  1.29185  0.71955             0.71955  1.29185   
20:00:04             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:05             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:06             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:07             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:08             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:09             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:10             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:11             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:12             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:13             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:14             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:15             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:16             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:17             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:18             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:19             0.71950  1.29175  0.71950             0.71950  1.29175   
20:00:20             0.71945  1.29175  0.71945             0.71945  1.29175   
20:00:21             0.71950  1.29175  0.71950             0.71950  1.29175   
20:00:22             0.71950  1.29175  0.71950             0.71950  1.29175   
20:00:23             0.71950  1.29175  0.71950             0.71950  1.29175   
20:00:24             0.71950  1.29175  0.71950             0.71950  1.29175   
20:00:25             0.71950  1.29175  0.71950             0.71950  1.29175   
20:00:26             0.71950  1.29175  0.71950             0.71950  1.29175   
20:00:27             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:28             0.71955  1.29175  0.71955             0.71955  1.29175   
20:00:29             0.71955  1.29175  0.71955             0.71955  1.29175   
...                      ...      ...      ...                 ...      ...   
20:59:30             0.71975  1.29260  0.71975             0.71975  1.29260   
20:59:31             0.71975  1.29260  0.71975             0.71975  1.29260   
20:59:32             0.71975  1.29260  0.71975             0.71975  1.29260   
20:59:33             0.71975  1.29260  0.71975             0.71975  1.29260   
20:59:34             0.71975  1.29260  0.71975             0.71975  1.29260   
20:59:35             0.71975  1.29260  0.71975             0.71975  1.29260   
20:59:36             0.71975  1.29260  0.71975             0.71975  1.29260   
20:59:37             0.71975  1.29260  0.71975             0.71975  1.29260   
20:59:38             0.71975  1.29260  0.71975             0.71975  1.29260   
20:59:39             0.71985  1.29265  0.71985             0.71985  1.29265   
20:59:40             0.71985  1.29265  0.71985             0.71985  1.29265   
20:59:41             0.71975  1.29265  0.71975             0.71975  1.29265   
20:59:42             0.71975  1.29265  0.71975             0.71975  1.29265   
20:59:43             0.71975  1.29265  0.71975             0.71975  1.29265   
20:59:44             0.71975  1.29265  0.71975             0.71975  1.29265   
20:59:45             0.71975  1.29265  0.71975             0.71975  1.29265   
20:59:46             0.71975  1.29265  0.71975             

In [257]:
p2 = list_of_hourly_mp[2][pd.Timestamp(non_duplicate_dates[1])]['6BU8']

In [295]:
p1.iat[3]

0.71955

In [296]:

(((p1.iat[i+1] - p1.iat[i])*1) - ((p2.iat[i+1] - p2.iat[i])*1)) +0

0.0

In [306]:
aux.find_spread(p1,p2,values)

size: 3600
3600
i: 3599


7200      0.000
7201      0.000
7202      0.000
7203      0.000
7204      6.250
7205      6.250
7206      6.250
7207      6.250
7208      6.250
7209      6.250
7210      6.250
7211      6.250
7212      6.250
7213      6.250
7214      6.250
7215      6.250
7216      6.250
7217      6.250
7218      6.250
7219      1.250
7220     -3.750
7221      1.250
7222      1.250
7223      1.250
7224      1.250
7225      1.250
7226      1.250
7227      6.250
7228      6.250
7229      6.250
          ...  
10770   -26.875
10771   -26.875
10772   -26.875
10773   -26.875
10774   -26.875
10775   -26.875
10776   -26.875
10777   -26.875
10778   -26.875
10779   -20.000
10780   -20.000
10781   -30.000
10782   -30.000
10783   -30.000
10784   -30.000
10785   -30.000
10786   -30.000
10787   -30.000
10788   -30.000
10789   -30.000
10790   -30.000
10791   -30.000
10792   -30.000
10793   -30.000
10794   -30.000
10795   -30.000
10796   -30.000
10797   -30.000
10798   -30.000
10799   -25.000
Name: spread, Length: 36

In [313]:
a= list_of_hourly_mp[0].set_index(list_of_hourly_mp[0].time,drop=True)

In [314]:
a[pd.Timestamp(non_duplicate_dates[0])]['6AU8']

time
18:00:00    0.71930
18:00:01    0.71930
18:00:02    0.71925
18:00:03    0.71925
18:00:04    0.71925
18:00:05    0.71935
18:00:06    0.71935
18:00:07    0.71935
18:00:08    0.71935
18:00:09    0.71935
18:00:10    0.71935
18:00:11    0.71935
18:00:12    0.71935
18:00:13    0.71935
18:00:14    0.71935
18:00:15    0.71935
18:00:16    0.71935
18:00:17    0.71935
18:00:18    0.71935
18:00:19    0.71935
18:00:20    0.71935
18:00:21    0.71935
18:00:22    0.71935
18:00:23    0.71935
18:00:24    0.71935
18:00:25    0.71935
18:00:26    0.71935
18:00:27    0.71935
18:00:28    0.71935
18:00:29    0.71935
             ...   
18:59:30    0.71925
18:59:31    0.71925
18:59:32    0.71925
18:59:33    0.71925
18:59:34    0.71925
18:59:35    0.71925
18:59:36    0.71925
18:59:37    0.71925
18:59:38    0.71925
18:59:39    0.71925
18:59:40    0.71925
18:59:41    0.71925
18:59:42    0.71925
18:59:43    0.71925
18:59:44    0.71925
18:59:45    0.71925
18:59:46    0.71925
18:59:47    0.71925
18:59:48    0.7

----

Burda spread hesaplanmıştır. Yeni yapı aşağıdaki gibidir: <br/>
```py
{'first_name':str, 
'second_name':str,
'date':datetime.date,
'first_product':list(pd.DataFrame,pd.DataFrame...),
'second_product':list(pd.DataFrame,pd.DataFrame...)
'spread': list(pd.Series,pd.Series....)}
```

In [216]:
values={'a_PNLTICK':10,
'a_TICKSIZE':0.0001,
'b_PNLTICK':6.25,
'b_TICKSIZE':0.0001}

In [43]:
eslestirilmis_urunler[0].keys()

dict_keys(['date', 'first_name', 'second_name', 'first_product', 'second_product', 'spread', 'change'])

---

4.  **daily_datas**: GünlükVeri örneklerinin tutulduğu liste <br />
**dt.to_match_hour** metodu ile her bir eşleşmiş çiftin mid_price'ları saatlik olarak alınıp birleştiriliyor. <br />
**gunluk_veri**: **tarih**, her bir saate ait **change**,**spread**,**amplitude**,**duration**,**urun adları**...vs içeriyor <br />
**paired_hours:** 23 adet df içerir. her bir df mid_price çifti içerir

In [ ]:
daily_data = []

In [ ]:
for pair in paired_days:
    product = pair[0].product + "_" + pair[1].product   # urun adi
    paired_hours=dt.to_match_hour(pair[0],pair[1])
    gunluk_veri = dt.DailyData(mid_price_list=paired_hours,date=pair[0].date, name= product)
    gunluk_veri.spread_hesapla()
    daily_data.append(gunluk_veri)

5. Her bir **DailyData** örneği ürünlere göre ayrı ayrı gruplandırılıyor. <br>
    (['**6AU8_6BU8**': [gunluk veriler...], '**6AU8_6CU8**':[gunluk veriler...], '**6BU8_6CU8**':[gunluk veriler...]])

In [ ]:
products={}

In [ ]:
for gunluk in daily_data:
    if gunluk.name in products:
        products[gunluk.name].append(gunluk)
    else:
        products[gunluk.name] = [gunluk]

**combined_days :**  key-value olarak ürün adları ve verileri içerir <br />
**combined_days_detail :** key-value olarak ürün adları ve verilere ait istatistiksel bilgileri içerir <br />
**combined_days_con :** key-value olarak ürün adları ve median koşuluna göre yeniden düzenlenmiş verileri içerir <br />
* 6BU8_6CU8'e ait verilere erişmek için:<br />
 ```python
combined_days_detail['6BU8_6CU8']
```
* 6BU8_6CU8'e ait 18:00:00-19:00:00 saatleri arası verilere erişmek için: <br />
 ```python
combined_days['6BU8_6CU8'].loc[time(18)]
```

In [ ]:
combined_days = {}
combined_days_detail = {}
combined_days_con = {}
combined_days_con_detail = {}

In [ ]:
stop=False
i=0
while products:
    urun = products.popitem()
    df = dt.get_all_data(urun[1]).dropna()
    con_df = con.scan(df)
    combined_days[urun[0]] = df
    combined_days_con[urun[0]] = con_df
#    combined_days_detail[urun[0]] = aux.get_detail(df)
#     combined_days_con_detail[urun[0]] = aux.get_detail(con_df)

In [ ]:
a = [1,2,3,4]

Hesaplanmış ürün adları ve saat bilgisi alınıyor

In [ ]:
all_products = list(combined_days.keys())

In [ ]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

all_density:
```json
{"6BU8_6CU8": {"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 "6AU8_6CU8":{"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 ....}
```

In [ ]:
all_density= {}

In [ ]:
for product in all_products:
    densities = {}
    for hour in hour_series:
        df = combined_days[product].loc[hour]
        densities[hour]= Density(df)
    all_density[product] = densities

--------------------------------

# Example

In [ ]:
data_of_the_eighteenth_clock = all_density['6AU8_6BU8'][time(18)]

data_of_the_eighteenth_clock.data

data_of_the_eighteenth_clock.density_data

In [ ]:
data_of_the_eighteenth_clock.draw()